In [2]:
from sqlalchemy import create_engine
import pandas as pd
import folium
from branca.colormap import linear
import pandas_scripts.locatie_coords_geoshape_script as lcs

In [3]:
DB_USER = "s1144085"
DB_PASS = "9515"
DB_HOST = "95.217.3.61"
DB_PORT = "5432"
DB_NAME = "minor_s1144085"
SCHEMA_NAME = "INMINDAA_project"

In [4]:
# Legt de verbinding met de database
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [5]:
# De functies om de data uit de database te halen
def fetch_leefbarometer_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_leefbarometer"}
    """
    return pd.read_sql(query, con=engine)

def fetch_armoede_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_armoede"}
    """
    return pd.read_sql(query, con=engine)

def fetch_hittestress_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_hittestress"}
    """
    return pd.read_sql(query, con=engine)

def fetch_huisartsen_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_huisartsen"}
    """
    return pd.read_sql(query, con=engine)

# Pakt alleen de data waar de leeftijdsgroep = '20 tot 45 jaar' en
# Waar de diagnose = '15.2 Ov. zwangerschap eind. misgeboorte'
def fetch_zwangerschappen_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}"."dim_zwangerschappen"
    WHERE diagnose = '15.2 Ov. zwangerschap eind. misgeboorte'
        AND leeftijdsgroep = '20 tot 45 jaar'
    """
    return pd.read_sql(query, con=engine)

In [6]:
# Data ophalen
df_leef = fetch_leefbarometer_data()
df_armoede = fetch_armoede_data()
df_hittestress = fetch_hittestress_data()
df_huisartsen = fetch_huisartsen_data()
df_zwangerschappen = fetch_zwangerschappen_data()

geodf = lcs.get_geodataframe()

te_verwijderen_score = [2.9613476]
df_leef = df_leef[~df_leef['lbm_score'].isin(te_verwijderen_score)]

Empty DataFrame
Columns: [pc4_code, Geo Shape]
Index: []


In [7]:
def maak_heatmap(geodf, df_data, value_column, tooltip_fields, titel, output_file):
    df_data['pc4_code'] = df_data['pc4_code'].astype(str)
    geodf['pc4_code'] = geodf['pc4_code'].astype(str)

    # Pakt het gemiddelde van de data per unieke pc4_code
    df_agg = df_data.groupby("pc4_code", as_index=False)[value_column].mean()

    # Merged de data met de geometrie
    geodf_merged = geodf.merge(df_agg, on='pc4_code', how='left')
    geodf_merged = geodf_merged.dropna(subset=[value_column])
    geodf_merged[value_column] = geodf_merged[value_column].astype(float)

    min_score = geodf_merged[value_column].min()
    max_score = geodf_merged[value_column].max()
    colormap = linear.YlOrRd_09.scale(min_score, max_score)
    geodf_dict = geodf_merged.set_index("pc4_code")[value_column]

    # Maakt de Folium-kaart met behulp van de geodata
    nl_map = folium.Map(location=[52.132633, 5.291266], zoom_start=8, tiles='Cartodb Positron')
    folium.GeoJson(
        geodf_merged,
        style_function=lambda feature: {
            "fillColor": colormap(geodf_dict.get(feature["properties"]["pc4_code"], 0)),
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=tooltip_fields),
    ).add_to(nl_map)

    colormap.caption = titel
    colormap.add_to(nl_map)

    nl_map.save(output_file)
    print(f"Kaart opgeslagen als {output_file}")

# Roept de functie op om de verschillende heatmaps te maken
maak_heatmap(geodf, df_leef, "lbm_score", ["pc4_code", "lbm_score"], "Leefbaarometer Score", "leefbaarometer_heatmap.html")
maak_heatmap(geodf, df_armoede, "g_ink_pi", ["pc4_code", "g_ink_pi"], "Gemiddeld Inkomen per Inwoner * 1000", "armoede_heatmap.html")
maak_heatmap(geodf, df_hittestress, "pet_gem", ["pc4_code", "pet_gem"], "Gemiddelde Gevoelstemperatuur tijdens Hittestress", "hittestress_heatmap.html")
maak_heatmap(geodf, df_huisartsen, "per_1e4", ["pc4_code", "per_1e4"], "Huisartsen Dichtheid per 10000 Inwoners", "huisartsen_heatmap.html")
maak_heatmap(geodf, df_zwangerschappen, "opnamen_per_1000_inwoners", ["pc4_code", "opnamen_per_1000_inwoners"], "Misgeboortes per 1000 vrouwen", "zwangerschap_heatmap_20tm45.html")

Kaart opgeslagen als zwangerschap_heatmap_20tm45.html
